#### Aula 2: Armazenamento Vetorial FAISS/Chroma

In [ ]:
# !pip install faiss-cpu chromadb langchain-pinecone pinecone-cliente
# !pip install langchain-huggingface sentence-transformers

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document

embeddings = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

In [ ]:
embeddings.embed_query("Política de home office da empresa")

In [ ]:
documentos_empresa = [
    Document(
        page_content="Política de férias: Funcionários têm direito a 30 dias de férias após 12 meses. A solicitação deve ser feita com 30 dias de antecedência.",
        metadata={"tipo": "política", "departamento": "RH", "ano": 2024, "id_doc": "doc001"}
    ),

    Document(
        page_content="Processo de reembolso de despesas: Envie a nota fiscal pelo portal financeiro. O reembolso ocorre em até 5 dias úteis.",
        metadata={"tipo": "processo", "departamento": "Financeiro", "ano": 2023, "id_doc": "doc002"}
    ),

    Document(
        page_content="Guia de TI: Para configurar a VPN, acesse vpn.nossaempresa.com e siga as instruções para seu sistema operacional.",
        metadata={"tipo": "tutorial", "departamento": "TI", "ano": 2024, "id_doc": "doc003"}
    ),

    Document(
        page_content="Código de Ética e Conduta: Valorizamos o respeito, a integridade e a colaboração. Casos de assédio não serão tolerados.",
        metadata={"tipo": "política", "departamento": "RH", "ano": 2022, "id_doc": "doc004"}
    )

]

#### Aula 3: FAISS

In [ ]:
from langchain_community.vectorstores import FAISS
import faiss

In [ ]:
d = 768
index_hnsw = faiss.IndexHNSWFlat(d, 32)

In [ ]:
faiss_db = FAISS.from_documents(documentos_empresa, embeddings)

pergunta = "Como peço minhas férias?"
resposta = faiss_db.similarity_search(pergunta, k=2)

In [ ]:
print(f"\nPergunta: {pergunta}")
print(f" Documentos mais relevantes (FAISS):")
for doc in resposta:
    print(f"- {doc.page_content}")
    print(f"  Metadados: { doc.metadata}")

#### Aula 3: ChromaDB

In [ ]:
from langchain_community.vectorstores import Chroma

chroma_db = Chroma.from_documents(
    documents=documentos_empresa,
    embedding=embeddings
)

In [ ]:
pergunta = "Como peço minhas férias?"
resultados = chroma_db.similarity_search(pergunta, k=2)

for doc in resultados:
    print(f"- {doc.page_content}")
    print(f"- {doc.metadata}\n")

In [ ]:
pergunta_rh = "Quais são as regras da empresa?"

resultados_filtrados = chroma_db.similarity_search(
    pergunta_rh,
    k=2,
    filter={"$and": [{"departamento": "RH"}, {"tipo": "política"}]}
)

In [ ]:
for doc in resultados_filtrados:
    print(f"- {doc.page_content}")
    print(f"- {doc.metadata}\n")

#### Aula 3: Pinecone

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

documentos_empresa = [
    Document(
        page_content="Política de férias: Funcionários têm direito a 30 dias de férias após 12 meses. A solicitação deve ser feita com 30 dias de antecedência.",
        metadata={"tipo": "política", "departamento": "RH", "ano": 2024, "id_doc": "doc001"}
    ),

    Document(
        page_content="Processo de reembolso de despesas: Envie a nota fiscal pelo portal financeiro. O reembolso ocorre em até 5 dias úteis.",
        metadata={"tipo": "processo", "departamento": "Financeiro", "ano": 2023, "id_doc": "doc002"}
    ),

    Document(
        page_content="Guia de TI: Para configurar a VPN, acesse vpn.nossaempresa.com e siga as instruções para seu sistema operacional.",
        metadata={"tipo": "tutorial", "departamento": "TI", "ano": 2024, "id_doc": "doc003"}
    ),

    Document(
        page_content="Código de Ética e Conduta: Valorizamos o respeito, a integridade e a colaboração. Casos de assédio não serão tolerados.",
        metadata={"tipo": "política", "departamento": "RH", "ano": 2022, "id_doc": "doc004"}
    )

]

d = 768

In [ ]:
from langchain_pinecone import Pinecone
from pinecone import Pinecone as PineconeClient
from pinecone import ServerlessSpec
import os

index_name = "langchain-rag"

pinecone_client = PineconeClient(api_key=os.environ["PINECONE_API_KEY"])

spec = ServerlessSpec(cloud="aws", region="us-east-1")

In [ ]:
# Verificando se índice existe, ou criá-lo.
if index_name not in pinecone_client.list_indexes().names():
    pinecone_client.create_index(
        name=index_name,
        dimension=d,
        metric="cosine",
        spec=spec
    )
    print(f"Indice '{index_name}' criado com sucesso.")

    print(f"Conectando a '{index_name}'")
    pinecone_db = Pinecone.from_documents(
        documentos_empresa,
        embeddings,
        index_name=index_name
    )

else:
    # Carregamento de índice existente
    print(f"Conectando a '{index_name}'")
    pinecone_db = Pinecone.from_existing_index(
        index_name,
        embeddings
    )


In [ ]:
print("Apagando dados duplicados...")
pinecone_db.delete(delete_all=True)

print(f"Enviando {len(documentos_empresa)} documentos para o Pinecone...")
pinecone_db.add_documents(documentos_empresa)

if pinecone_db:
    # Busca por similaridade
    pergunta_ti = "Como configurar VPN?"
    resultados_pinecone = pinecone_db.similarity_search(pergunta_ti, k=2)
    print(f"\nPergunta: {pergunta_ti}")
    print(f" Documentos mais relevantes (Pinecone):")
    for doc in resultados_pinecone:
        print(f"- {doc.page_content}")
        print(f"  Metadados: { doc.metadata}")
    print("\n")

    # Busca com filtro
    resultados_pinecone_filtrados = pinecone_db.similarity_search(
        "Informações sobre regras", k=2,
        filter={"tipo": "política"}
    )
    print(f"\nPergunta: Informações sobre regras")
    print(f" Documentos mais relevantes (Pinecone):")
    for doc in resultados_pinecone_filtrados:
        print(f"- {doc.page_content}")
        print(f"  Metadados: { doc.metadata}")